# WAVSS

### Purpose
The purpose of this notebook is to calculate the gross range user min/max values for populating QARTOD parameter tables for data streams for OOI - CGSN data streams for the TRIAXYS Nex Wave (WAVSS) sensor. The WAVSS measures three-dimensional acceleration, rotation rate, and orientation in the Earth's magnetic field. 

The WAVSS datasets are split into five types of datastreams: statistics, motion, fourier, non-directional, and mean directional. The motion datastreams contain the buoy displacement values (north, east, heave). The fourier datastreams contain the fourier coefficients computed for both the directional and non-directional wave spectra, along with associated frequencies, frequency spacing, number of bands, etc. The non-directional datastreams contain the PSDs and associated frequencies used to compute the bulk wave statistics. The mean directional datastreams contain the parameters for mean direction of the wave field, the spread direction, and the PSDs with associated frequencies used to compute those parameters. The statistics datastreams contain the bulk wave parameters: significant wave height; peak wave period; mean wave direction; wave spread; and other associated parameters. 

It is the _bulk_wave_statistics_ datastream that we are interested in testing, since that has the science-relevant parameters.
  
### Test Parameters

| Variable | Description | Range | Motion or Spectral Product |
|----------|-------|------------|----------|
| average_wave_height | Average zero down-crossing wave height (m) | [0, 40] | Motion |
| max_wave_height | Maximum zero down-crossing wave height (trough to peak) (m) | [0, 40] | Motion |
| mean_direction | mean direction of wave field against true north | [0, 360] | Directional |
| mean_spectral_period | | [1.5, 33] | Directional |
| mean_spread | Mean directional spread of wave field (units: degrees) | [0, 90] | Directional |
| mean_wave_period | average wave period | [1.5, 33] | Motion |
| number_zero_crossings | Number of zero crossings in the underlying displacement data, used primarily for verification and QC. | NA | Motion |
| peak_wave_period | Peak wave period; period where wave spectrum has its maximum | [1.5, 33] | Directional |
| significant_wave_period | Significant wave period, average period of the Hsig waves. | [1.5, 33] | Motion |
| significant_wave_height | Significant wave height, average height of the highest third of the waves (Hsig) | [0, 40] | Motion |
| wave_height_10 | Average height of the highest tenth (H10) of the waves | [0, 40] | Motion |
| wave_height_hmo | Significant wave height estimated by an alternate method based on spectral moments (zeroth moment method) | [0, 40] | Directional |
| wave_period_10 | Average period of the H10 waves | [1.5, 33] | Motion |
| wave_period_tp5 | Peak wave period computed via Read method; an alternative to TP | [1.5, 33] | Directional |



#### Import libraries

In [ ]:
import os, sys, datetime, pytz, re, yaml
import dateutil.parser as parser
import pandas as pd
import numpy as np
import xarray as xr
import warnings
import gc
import json
warnings.filterwarnings("ignore")

In [ ]:
from dask.diagnostics import ProgressBar

#### Import the ```ooinet``` M2M toolbox
This toolbox is publicly available at https://github.com/reedan88/OOINet. It should be cloned onto your machine and the setup instructions followed before use.

In [ ]:
# Import the OOINet M2M tool
sys.path.append("/home/areed/Documents/OOI/reedan88/ooinet/")
from ooinet import M2M
from ooinet.utils import convert_time, ntp_seconds_to_datetime, unix_epoch_time
from ooinet.Instrument.common import process_file, add_annotation_qc_flag

#### Import ```ooi_data_explorations``` toolbox
This toolbox is publicly available at https://github.com/oceanobservatories/ooi-data-explorations. Similarly to the ```ooinet``` toolbox above, it should be installed onto your machine following the setup instructions before use.

In [ ]:
sys.path.append("/home/areed/Documents/OOI/oceanobservatories/ooi-data-explorations/python/")
from ooi_data_explorations.common import get_annotations, get_vocabulary, load_gc_thredds
from ooi_data_explorations.combine_data import combine_datasets
from ooi_data_explorations.uncabled.process_wavss import wavss_datalogger
from ooi_data_explorations.qartod.qc_processing import identify_blocks, create_annotations, process_gross_range, \
    process_climatology, parse_qc, inputs, ANNO_HEADER, CLM_HEADER, GR_HEADER

#### Import plotting and visualization tools

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from ooi_data_explorations.qartod.plotting import *

**Import the test parameters with ranges**

In [ ]:
with open("../data/sensor_ranges.yaml") as stream:
    sensor_ranges = yaml.safe_load(stream)

---
## Identify Data Streams
This section is necessary to identify all of the data stream associated with a specific instrument. This can be done by querying UFrame and iteratively walking through all of the API endpoints. The results are saved into a csv file so this step doesn't have to be repeated each time.

First, set the instrument to search for using OOI terminology:

In [ ]:
instrument = "WAVSS"

### Query OOINet for Data Streams <br>
First check if the datasets have already been downloaded; if not, use the ```M2M.search_datasets``` tool to search the OOINet API and return a table of all of the available datasets for the given instruments.

In [ ]:
try:
    datasets = pd.read_csv("../data/WAVSS_datasets.csv")
except:
    datasets = M2M.search_datasets(instrument="WAVSS", English_names=True)
    # Save the datasets
    datasets.to_csv("../data/WAVSS_datasets.csv", index=False)

In [ ]:
datasets

Separate out the CGSN datasets from the EA datasets:

In [ ]:
cgsn = datasets["array"].apply(lambda x: True if x.startswith(("CP","GA","GI","GP","GS")) else False)
datasets = datasets[cgsn]
datasets

---
## Single Reference Designator
The reference designator acts as a key for an instrument located at a specific location. First, select a reference designator (refdes) to request data from OOINet.

In [ ]:
reference_designators = sorted(datasets["refdes"])
print("Number of reference designators: " + str(len(reference_designators)))
for refdes in reference_designators:
    print(refdes)

In [ ]:
k=3
refdes = reference_designators[k]
print(refdes)

#### Sensor Vocab
The vocab provides information about the instrument model and type, its location (with descriptive names), depth, and manufacturer. Get the vocab for the given reference designator.

In [ ]:
vocab = M2M.get_vocab(refdes)
vocab

#### Sensor Deployments
Download the deployment information for the selected reference designator:

In [ ]:
deployments = M2M.get_deployments(refdes)
deployments

#### Sensor Data Streams
Next, select the specific data streams for the given reference designator

In [ ]:
datastreams = M2M.get_datastreams(refdes)
datastreams

---
## Metadata 
The metadata contains the following important key pieces of data for each reference designator: **method**, **stream**, **particleKey**, and **count**. The method and stream are necessary for identifying and loading the relevant dataset. The particleKey tells us which data variables in the dataset we should be calculating the QARTOD parameters for. The count lets us know which dataset (the recovered instrument, recovered host, or telemetered) contains the most data and likely has the best record to use to calculate the QARTOD tables. 

In [ ]:
metadata = M2M.get_metadata(refdes)
metadata

#### Sensor Parameters
Each instrument returns multiple parameters containing a variety of low-level instrument output and metadata. However, we are interested in science-relevant parameters for calculating the relevant QARTOD test limits. We can identify the science parameters based on the preload database, which designates the science parameters with a "data level" of L1 or L2. 

Consequently, we through several steps to identify the relevant parameters. First, we query the preload database with the relevant metadata for a reference designator. Then, we filter the metadata for the science-relevant data streams. 

In [ ]:
def filter_science_parameters(metadata):
    """This function returns the science parameters for each datastream"""
    
    def filter_parameter_ids(pdId, pid_dict):
        data_level = pid_dict.get(pdId)
        if data_level is not None:
            if data_level > 0:
                return True
            else:
                return False
        else:
            return False
    
    # Filter the parameters for processed science parameters
    data_levels = M2M.get_parameter_data_levels(metadata)
    mask = metadata["pdId"].apply(lambda x: filter_parameter_ids(x, data_levels))
    metadata = metadata[mask]

    return metadata

def filter_metadata(metadata):
    science_vars = filter_science_parameters(metadata)
    # Next, eliminate the optode temperature from the stream
    mask = science_vars["particleKey"].apply(lambda x: False if "temp" in x else True)
    science_vars = science_vars[mask]
    science_vars = science_vars.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
    science_vars = science_vars.reset_index()
    science_vars = science_vars.applymap(lambda x: x[0] if len(x) == 1 else x)
    science_vars = science_vars.explode(column="particleKey")
    return science_vars

In [ ]:
#science_vars = filter_science_parameters(metadata)
science_vars = metadata.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
science_vars = science_vars.reset_index()
science_vars = science_vars.applymap(lambda x: x[0] if len(x) == 1 else x)
science_vars

**Filter data streams for bulk wave statistics**<br>
The WAVSS returns a lot of different data products, including spectral products, fourier series, and raw motion data. We are interested in testing _only_ the bulk wave statistics datastreams, which have the relevant science wave parameters such as signficant wave height, wave period, etc.

In [ ]:
mask = science_vars["stream"].apply(lambda x: True if "stat" in x else False)
science_vars = science_vars[mask]
science_vars

---
## Load Data
When calculating the QARTOD data tables, we only want to utilize the most complete data record available for a given reference designator. We do this by getting all the available data streams, loading the data, and then combining them into a single dataset. 

In [ ]:
def trim_overlaps(ds, deployments):
    """Trim overlapping deployment data (necessary to use xr.open_mfdataset)"""
    # --------------------------------
    # Second, get the deployment times
    deployments = deployments.sort_values(by="deploymentNumber")
    deployments = deployments.set_index(keys="deploymentNumber")
    # Shift the start times by (-1) 
    deployEnd = deployments["deployStart"].shift(-1)
    # Find where the deployEnd times are earlier than the deployStart times
    mask = deployments["deployEnd"] > deployEnd
    # Wherever the deployEnd times occur after the shifted deployStart times, replace those deployEnd times
    deployments["deployEnd"][mask] = deployEnd[mask]
    deployments["deployEnd"] = deployments["deployEnd"].apply(lambda x: pd.to_datetime(x))
    
    # ---------------------------------
    # With the deployments info, can write a preprocess function to filter 
    # the data based on the deployment number
    depNum = np.unique(ds["deployment"])
    deployInfo = deployments.loc[depNum]
    deployStart = deployInfo["deployStart"].values[0]
    deployEnd = deployInfo["deployEnd"].values[0]
    
    # Select the dataset data which falls within the specified time range
    ds = ds.sel(time=slice(deployStart, deployEnd))
    
    return ds

def preprocess(ds):
    ds = process_file(ds)
    ds = wavss_datalogger(ds)
    ds = trim_overlaps(ds, deployments)
    gc.collect()
    return ds

#### Identify the bulk wave statistics datastreams

In [ ]:
mask = datastreams["stream"].apply(lambda x: True if "stat" in x else False)
datastreams[mask]

In [ ]:
# Get the available datasets
for index in datastreams[mask].index:
    # Get the method and stream
    method = datastreams.loc[index]["method"]
    stream = datastreams.loc[index]["stream"]

    # Get the URL - first try the goldCopy thredds server
    thredds_url = M2M.get_thredds_url(refdes, method, stream, goldCopy=True)

    # Get the catalog
    catalog = M2M.get_thredds_catalog(thredds_url)

    # Clean the catalog
    catalog = M2M.clean_catalog(catalog, stream, deployments)
    
    # Get the links to the THREDDs server and load the data
    dodsC = M2M.URLS["goldCopy_dodsC"]
    if method == "telemetered":
        tele_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            tele_data = xr.open_mfdataset(tele_files, preprocess=preprocess, parallel=True)
    elif method == "recovered_host":
        host_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            host_data = xr.open_mfdataset(host_files, preprocess=preprocess, parallel=True)
    else:
        pass

**Combine the datasets into a single dataset**

In [ ]:
data = combine_datasets(tele_data, host_data, None, None)
data

---
## Process the WAVSS
The WAVSS data needs reprocessing before we can generate the qartod tables. This includes removing data identified by annotations as bad and filtering out data that, based on HITL review, is outside of the expected instrument calibration ranges. Once these steps are done, then the cleaned up datasets can be used to generate more-accurate QARTOD tables.

#### Annotations
The annotations associated with a specific reference designator may contain relevant information on the performance or reliability of the data for a given dataset. The annotations are downloaded from OOINet as a json and processed into a pandas dataframe. Each annotation may apply to the entire dataset, to a specific stream, or to a specific variable. With the downloaed annotations, we can use the information contained in the ```qcFlag``` column to translate the annotations into QC flags, which can then be used to filter out bad data. 

In [ ]:
annotations = M2M.get_annotations(refdes)
annotations

Look at some individual annotations

In [ ]:
annotations.loc[2]["annotation"]

Pass in the annotations and the dataset to add the annotation ```qcFlag``` values to the dataset

In [ ]:
data = add_annotation_qc_flag(data, annotations)
data

Plot some of the data with the annotations on top

In [ ]:
def plot_variable(ds, param, add_deployments=True):
    """Function to plot the timeseries with deployment info.

    Parameters
    ----------
    ds: (xarray.Dataset)
        An xarray dataset downloaded from OOINet
    param: (str)
        The parameter name of the data variable in the OOI
        dataset to plot
    add_deployments: (boolean)
        Also plot deployment information

    Returns
    -------
    fig, ax: (matplotlib figs)
        Figure and axis handles for the matplotlib image
    """
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Calculate the figure bounds
    yavg = ds[param].mean()
    ystd = ds[param].std()
    ymed = np.nanmedian(ds[param])
    # Need to check for way out-of-bounds values by comparison with median
    if ystd > ymed:
        yavg = ymed
        ystd = ymed*0.2
    ymin = yavg - 4*ystd
    ymax = yavg + 4*ystd
    
    # Generate the plot figure
    if add_deployments:
        s = ds.plot.scatter("time", param, ax=ax, hue="deployment", hue_style="discrete")
    else:
        s = ds.plot.scatter("time", param, ax=ax)
        
    # Add in limits and labels
    #ax.set_ylim((ymin, ymax))
    xlabel = ax.get_xlabel()
    ax.set_xlabel(xlabel, fontsize=14)
    ax.set_ylabel(ds[param].attrs["long_name"], fontsize=14)
    ax.set_title(ds.attrs["id"], fontsize=16)
    ax.grid()
    
    # Add in legend if deployments added
    if add_deployments:
        ax.legend(edgecolor="black", loc="center left", bbox_to_anchor=(1, 0.5), fontsize=12, title="Deployments")
        deployments = np.unique(ds["deployment"])
        for depNum in deployments:
            dt = ds.where(ds["deployment"] == depNum, drop=True)["time"].min()
            ax.vlines(dt.values, yavg-4*ystd, yavg+4*ystd)
            ax.text(dt.values, yavg-3*ystd, str(int(depNum)), fontsize=14, weight="bold")
            
    fig.autofmt_xdate()
    
    return fig, ax

In [ ]:
sensor_ranges

In [ ]:
data.load()

In [ ]:
param = "wave_height_hmo"

fig, ax = plot_variable(data, param)

# Plot the annotations
pqc = param + "_qc_summary_flag"
qc3 = (data[pqc] == 3)
qc4 = (data[pqc] > 3)
ax.plot(data.time[qc3], data[param][qc3], marker=".", linestyle="", color="yellow")
ax.plot(data.time[qc4], data[param][qc4], marker=".", linestyle="", color="tab:red")

Use the added ```rollup_annotations_qc_results``` values to filter out bad or suspect (```rollup_annotations_qc_results``` value of 3, 4, or 9) data from the dataset

In [ ]:
data = data.where(data.rollup_annotations_qc_results <= 3, drop=True)
data

#### Limit the data to data collected before 2021-01-01

In [ ]:
_, index = np.unique(data['time'], return_index=True)
data = data.isel(time=index)
data = data.sel(time=slice('2014-01-01T00:00:00', "2023-01-01T00:00:00"))
data

---
## Gross Range
The Gross Range QARTOD test consists of two parameters: a fail range which indicates when the data is bad, and a suspect range which indicates when data is either questionable or interesting. The fail range values are set based upon the instrument/measurement and associated calibration. For example, the conductivity sensors are calibration for measurements between 0 (freshwater) and 9 (highly-saline waters). The suspect range values are calculated based on the mean of the available data $\pm$3$\sigma$.

In [ ]:
from ooi_data_explorations.qartod.gross_range import GrossRange
from ooi_data_explorations.qartod.plotting import *
from ooi_data_explorations.qartod.qc_processing import format_gross_range, format_climatology

#### Test Parameters & Sensor Ranges
The sensor ranges are stored in a yaml file. Load those and get the test parameters

In [ ]:
test_parameters = sensor_ranges["test_parameters"]
test_parameters

**Calculate the Gross Range Values**

In [ ]:
site, node, sensor = refdes.split("-", 2)
gross_range_table = pd.DataFrame()

for param in test_parameters:
    sensor_range = test_parameters.get(param)
    
    # Set the input parameter name
    inp = param
    
    # Clean up the parameter based on the QC flag
    param_qc = param + "_qc_summary_flag"
    
    if param in data.variables:
        print(f"##### Calculating gross range for {param} #####")
        param_data = data.where((data[param_qc] < 3), drop=True)
        # Check if there is enough data
        if len(param_data[param].dropna(dim="time")) < 100:
            user_range = [np.nan, np.nan]
            source = "Not enough data to calculate user range."
        else:
            gross_range = GrossRange(sensor_range[0], sensor_range[1])
            gross_range.fit(param_data, param, check_normality=True)
            user_range = [gross_range.suspect_min, gross_range.suspect_max]
            source = gross_range.source
        # Check which streams have the param in it
        streams = metadata[metadata["particleKey"] == inp]["stream"].unique()
        for stream in streams:
            qc_dict = format_gross_range(inp, sensor_range, user_range, site, node, sensor, stream, source)
            gross_range_table = gross_range_table.append(qc_dict, ignore_index=True)
            
        # Plot the result
        try:
            fig, ax = plot_gross_range(param_data, param, gross_range)
            ymin, ymax = ax.get_ylim()
            if ymin < sensor_range[0]:
                ymin = sensor_range[0]
            if ymax > sensor_range[1]:
                ymax = sensor_range[1]
            ax.set_ylim((ymin, ymax))
            #ax.set_ylim((200, 600))
        except:
            pass

**Add the stream name and the source comments**

In [ ]:
gross_range_table['notes'] = ('User range based on data collected through {}.'.format("2021-01-01"))
gross_range_table

**Check the results**

In [ ]:
for ind in gross_range_table.index:
    print(gross_range_table.loc[ind]["qcConfig"])

**Save the gross range table**

In [ ]:
# Save the results
gross_range_table.to_csv(f"../results/gross_range/{refdes}.csv", index=False, columns=GR_HEADER)

---
## Climatology
For the climatology QARTOD test, First, we bin the data by month and take the mean. The binned-montly means are then fit with a 2 cycle harmonic via Ordinary-Least-Squares (OLS) regression. Ranges are calculated based on the 3$\sigma$ calculated from the OLS-fitting.  

In [ ]:
from ooi_data_explorations.qartod.climatology import Climatology

In [ ]:
def make_climatology_table(ds, param, tinp, zinp, sensor_range, depth_bins):
    """Function which calculates the climatology table based on the """
    
    climatologyTable = pd.DataFrame()
    
    if depth_bins is None:
        # Filter out the data outside the sensor range
        m = (ds[param] > sensor_range[0]) & (ds[param] < sensor_range[1]) & (~np.isnan(ds[param]))
        param_data = ds[param][m]
        
        # Fit the climatology for the selected data
        pmin, pmax = [0, 0]
        
        try:
            climatology = Climatology()
            climatology.fit(param_data)

            # Create the depth index
            zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

            # Create the monthly bins
            tspan = pd.interval_range(0, 12, closed="both")

            # Calculate the climatology data
            vmin = climatology.monthly_fit - climatology.monthly_std*3
            vmin = np.floor(vmin*10000)/10000
            for vind in vmin.index:
                if vmin[vind] < sensor_range[0] or vmin[vind] > sensor_range[1]:
                    vmin[vind] = sensor_range[0]
            vmax = climatology.monthly_fit + climatology.monthly_std*3
            for vind in vmax.index:
                if vmax[vind] < sensor_range[0] or vmax[vind] > sensor_range[1]:
                    vmax[vind] = sensor_range[1]
            vmax = np.floor(vmax*10000)/10000
            vdata = pd.Series(data=zip(vmin, vmax), index=vmin.index).apply(lambda x: [v for v in x])
            vspan = vdata.values.reshape(1,-1)

            # Build the climatology dataframe
            climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

        except:
            # Here is where to create nans if insufficient data to fit
            # Create the depth index
            zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

            # Create the monthly bins
            tspan = pd.interval_range(0, 12, closed="both")

            # Create a series filled with nans
            vals = []
            for i in np.arange(len(tspan)):
                vals.append([np.nan, np.nan])
            vspan = pd.Series(data=vals, index=tspan).values.reshape(1,-1)

            # Add to the data
            climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))
            
        del ds, vspan, tspan, zspan
        gc.collect()
        
    else:        
    # Iterate through the depth bins to calculate the climatology for each depth bin
        for dbin in depth_bins:
            # Get the pressure range to bin from
            pmin, pmax = dbin[0], dbin[1]

            # Select the data from the pressure range
            bin_data = data.where((data[zinp] >= pmin) & (data[zinp] <= pmax), drop=True)

            # sort based on time and make sure we have a monotonic dataset
            bin_data = bin_data.sortby('time')
            _, index = np.unique(bin_data['time'], return_index=True)
            bin_data = bin_data.isel(time=index)

            # Filter out the data outside the sensor range
            m = (bin_data[param] > sensor_range[0]) & (bin_data[param] < sensor_range[1]) & (~np.isnan(bin_data[param]))
            param_data = bin_data[param][m]

            # Fit the climatology for the selected data
            try:
                climatology = Climatology()
                climatology.fit(param_data)

                # Create the depth index
                zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

                # Create the monthly bins
                tspan = pd.interval_range(0, 12, closed="both")

                # Calculate the climatology data
                vmin = climatology.monthly_fit - climatology.monthly_std*3
                vmin = np.floor(vmin*10000)/10000
                for vind in vmin.index:
                    if vmin[vind] < sensor_range[0] or vmin[vind] > sensor_range[1]:
                        vmin[vind] = sensor_range[0]
                vmax = climatology.monthly_fit + climatology.monthly_std*3
                vmax = np.floor(vmax*10000)/10000
                for vind in vmax.index:
                    if vmax[vind] < sensor_range[0] or vmax[vind] > sensor_range[1]:
                        vmax[vind] = sensor_range[1]
                vdata = pd.Series(data=zip(vmin, vmax), index=vmin.index).apply(lambda x: [v for v in x])
                vspan = vdata.values.reshape(1,-1)

                # Build the climatology dataframe
                climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

            except:
                # Here is where to create nans if insufficient data to fit
                # Create the depth index
                zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

                # Create the monthly bins
                tspan = pd.interval_range(0, 12, closed="both")

                # Create a series filled with nans
                vals = []
                for i in np.arange(len(tspan)):
                    vals.append([np.nan, np.nan])
                vspan = pd.Series(data=vals, index=tspan).values.reshape(1,-1)

                # Add to the data
                climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

            del climatology, bin_data, vspan, tspan, zspan
            gc.collect()
    
    return climatologyTable, climatology

**Get the depth bins and filter based on max depth.** <br>
For the ```WAVSS``` which are only deployed on Surface Moorings, there is no depth bins needed.

In [ ]:
depth_bins = None

In [ ]:
# Initialize the climatology lookup table
climatologyLookup = pd.DataFrame()

# Setup the Table Header
TBL_HEADER = ["[1,1]","[2,2]","[3,3]","[4,4]","[5,5]","[6,6]","[7,7]","[8,8]","[9,9]","[10,10]","[11,11]","[12,12]"]

# Set the subsite-node-sensor
subsite, node, sensor = refdes.split("-", 2)

# Iterate through the parameters
for param in test_parameters:
    # ----------------- Drop bad data from the parameter ---------------------
    param_qc = param + "_qc_summary_flag"
    param_data = data.where(data[param_qc] < 3, drop=True)
    
    if param in data.variables:
        inp = param
        # ----------------- Depth tables ---------------------
        # Get the sensor range of the parameter to test
        print(f"##### Calculating climatology for {param} #####")
        sensor_range = test_parameters.get(param)
        
        # Generate the climatology table with the depth bins
        climatologyTable, climatology = make_climatology_table(param_data, param, "time", "depth", sensor_range, depth_bins)
        
        # Get the variance
        try:
            variance = float(np.round(climatology.regression['variance_explained']*100, 1))
        except:
            variance = 0.0

        # Create the tableName
        tableName = f"{refdes}-{param}.csv"
        
        # Save the results
        climatologyTable.to_csv(f"../results/climatology/climatology_tables/{tableName}", header=TBL_HEADER)
        
        # ------------------ Lookup tables ------------------
        # Check which streams have the param in it
        streams = metadata[metadata["particleKey"] == inp]["stream"].unique()
        for stream in streams:
            qc_dict = {
                "subsite": subsite,
                "node": node,
                "sensor": sensor,
                "stream": stream,
                "parameters": {
                    "inp": inp,
                    "tinp": "time",
                    "zinp": "depth",
                },
                "climatologyTable": f"climatology_tables/{refdes}-{param}.csv",
                "source": f"The variance explained by the climatology model is {variance}%.",
                "notes": "Climatology based on available data through 2021-01-01."
            }
            # Append to the lookup table
            climatologyLookup = climatologyLookup.append(qc_dict, ignore_index=True)
            
        # ------------------ Plot the climatology ------------------
        if param_data[param].time.size > 100000:
            try:
                subset = sorted(np.random.choice(param_data.time, 100000, replace=False))
                subset_data = param_data.sel(time=subset)
                fig, ax = plot_climatology(subset_data, param, climatology)
                #ax.set_ylim((sensor_range[0], sensor_range[1]))
                del subset, subset_data
                gc.collect()
            except:
                pass
        else:
            try:
                fig, ax = plot_climatology(param_data, param, climatology)
                #ax.set_ylim((sensor_range[0], sensor_range[1]))
            except:
                pass

**Check the last climatologyTable for reasonableness**

In [ ]:
climatologyTable

**Check the climatologyLookup table that all the entries made it in**

In [ ]:
climatologyLookup

**Save the climatologyLookup table**

In [ ]:
# Save the lookup table results
climatologyLookup.to_csv(f"../results/climatology/{refdes}.csv", index=False, columns=CLM_HEADER)